In [1]:
import pandas as pd
import numpy as np

In [2]:
def feature_normalize(X):
    #   FEATURENORMALIZE Normalizes the features in X 
    #   FEATURENORMALIZE(X) returns a normalized version of X where
    #   the mean value of each feature is 0 and the standard deviation
    #   is 1. This is often a good preprocessing step to do when
    #   working with learning algorithms.

    # You need to set these values correctly
    X_norm = X
    mu     = np.zeros(X.shape[1])
    sigma  = np.zeros(X.shape[1])
    
    # ====================== YOUR CODE HERE ======================
    # Instructions: First, for each feature dimension, compute the mean
    #               of the feature and subtract it from the dataset,
    #               storing the mean value in mu. Next, compute the 
    #               standard deviation of each feature and divide
    #               each feature by it's standard deviation, storing
    #               the standard deviation in sigma. 
    #
    #               Note that X is a matrix where each column is a 
    #               feature and each row is an example. You need 
    #               to perform the normalization separately for 
    #               each feature. 
    #
    # Hint: You might find the 'np.mean' and 'np.std' functions useful.
    #  
    

    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    
    X_norm = (X - mu) / sigma
    
    
    # ============================================================
    
    return X_norm, mu, sigma

In [3]:
def normal_eqn(X, y):
    #NORMALEQN Computes the closed-form solution to linear regression 
    #   NORMALEQN(X,y) computes the closed-form solution to linear 
    #   regression using the normal equations.

    theta = np.zeros(X.shape[1])

    # ====================== YOUR CODE HERE ======================
    # Instructions: Complete the code to compute the closed form solution
    #               to linear regression and put the result in theta.
    #


    theta = np.dot(np.dot(np.linalg.pinv(np.dot(X.T, X)), X.T), y)
    # ============================================================
    return theta

In [4]:
def read_training_data(file_name, encoding='big5'):
    df = pd.read_csv(file_name, encoding=encoding, na_values=['NR'])
    df = df.fillna(0)
    return df

df = read_training_data('./train.csv')

In [5]:
def preprocess_training_data(df):
    df = df.rename(
        columns={
            df.columns[0]: 'date',
            df.columns[1]: 'location',
            df.columns[2]: 'feature'
    })
        
    df.date = pd.to_datetime(df.date)
    
    df = df.drop(columns=['location'])

    
    return df

df = preprocess_training_data(df)

In [6]:
data2 = df[df.feature == 'PM2.5']

In [7]:
data10 = df[df.feature == 'PM10']

In [8]:
flat_data2 = []
for name, group in data2.groupby(data2.date.dt.month):
    group = group.drop(columns=['date', 'feature'])
    flat_data2.append(group.values.flatten())
    
flat_data2 = np.array(flat_data2)

In [9]:
flat_data10 = []
for name, group in data10.groupby(data10.date.dt.month):
    group = group.drop(columns=['date', 'feature'])
    flat_data10.append(group.values.flatten())
    
flat_data10 = np.array(flat_data10)

In [12]:
X = []
y = []
for m2, m10 in zip(flat_data2, flat_data10):
#     print(m.shape)
    for i in range(m2.size-9):
        X.append(np.append(m2[i:i+9],m10[i:i+9]))
        y.append(m2[i+9])
X = np.array(X)
y = np.array(y).reshape((-1, 1))

In [13]:
X_n, mu, sigma = feature_normalize(X)

In [15]:
def read_testing_data(file_name):
    df = pd.read_csv(file_name, header=None, na_values=['NR'])
    df = df.rename(columns={0: 'id', 1: 'feature'})
    df = df.fillna(0)
    return df
tdf = read_testing_data('./test.csv')

In [16]:
tdf2 = tdf[tdf.feature == 'PM2.5']
tdf2 = tdf2.drop(columns=['id', 'feature'])
t2 = tdf2.values 

In [17]:
tdf10 = tdf[tdf.feature == 'PM10']
tdf10 = tdf10.drop(columns=['id', 'feature'])
t10 = tdf10.values

In [18]:
# tdf.index = tdf.id

In [19]:
# tdf = tdf.drop(columns=['id', 'feature'])

In [20]:
# test = tdf.values

In [21]:
theta = normal_eqn(np.insert(X_n, 0, 1, 1), y)

In [22]:
ans = pd.DataFrame(columns=['id', 'value'])

In [27]:
pred = []
for y2, y10 in zip(t2, t10):
#     print(y2, y10)
    t = (np.append(y2, y10)-mu)/sigma
    pred.append(np.dot(np.insert(t, 0, 1, 0), theta))

pred = np.array(pred)

In [36]:
ans.id = tdf.id.unique()

In [37]:
ans.value = pred.flatten()

In [38]:
ans.to_csv('final.csv', index=False)

In [22]:
def compute_cost_multi(theta, X, y):
    m, n = X.shape
    h_theta = np.dot(X, theta)
    square_error = (h_theta - y)**2
    J = 1 / (2 * m) * np.sum(square_error)
    return J

In [23]:
def gradient_descent_multi(X, y, theta, alpha, num_iters):
    # GRADIENTDESCENT Performs gradient descent to learn theta
    # theta = GRADIENTDESCENT(X, y, theta, alpha, num_iters) updates theta by 
    # taking num_iters gradient steps with learning rate alpha
    m, n = X.shape
    # Initialize
    J_history = np.zeros((num_iters, 1))
#     T_history = np.zeros((num_iters,X.shape[1], 1))
    
    for i in range(num_iters):
#         T_history[i] = theta

        ### ========= YOUR CODE HERE ============
        # Instructions: Perform a single gradient step on the parameter vector theta.
        
        htheta = np.dot(X, theta)
        theta = theta - (alpha / m) * np.dot(X.T, htheta-y)
        
        ### =====================================
        
        J_history[i] = compute_cost_multi(theta, X, y)
    return theta, J_history#, T_history


In [29]:
g_theta, J_his = gradient_descent_multi(np.insert(X_n, 0, 1, 1), y, theta, 0.01, 20000)